# Carga de datos

In [1]:
from mysql.connector import connect, Error
import pandas as pd

In [3]:
def Carga_df(NombreCSV, Sep):
    df = pd.read_csv(NombreCSV, sep=Sep, header=0)
    print(f"Lectura de {NombreCSV} terminada.")
    return df 

In [ ]:
BP_EstadoCliente = Carga_df('01_BP_EstadoCliente.csv', ',')

CLI_Cliente = Carga_df('02_CLI_Cliente.csv', ',')  

BP_TipoCuentaCobro = Carga_df('03_BP_TipoCuentaCobro.csv', ',')

BP_EstadoTipoCtaCobro = Carga_df('04_BP_EstadoTipoCtaCobro.csv', ',')   

BP_EstadoCuentaCobro = Carga_df('05_BP_EstadoCuentaCobro.csv', ',')

BP_EstadoServicio = Carga_df('06_BP_EstadoServicio.csv', ',')

BP_TipoServicio = Carga_df('07_BP_TipoServicio.csv', ',')

BP_EstadoServicio_BPTipoServicio = Carga_df('08_BP_EstadoServicio_BPTipoServicio.csv', ',')

BP_Moneda = Carga_df('09_BP_Moneda.csv', ',')

CTACTES_CuentaCobro = Carga_df('10_CTACTES_CuentaCobro.csv', ',')

BP_TipoMovimiento = Carga_df('11_BP_TipoMovimiento.csv', ',')

CTACTES_CuentaCobroMovimiento = Carga_df('12_CTACTES_CuentaCobroMovimiento.csv', ',')

COMERCIAL_ProductoComercial = Carga_df('21_COMERCIAL_ProductoComercial.csv', ',')

SERVICIOS_Servicio = Carga_df('13_SERVICIOS_Servicio.csv', ',')

BP_TipoRecursoTecnico = Carga_df('14_BP_TipoRecursoTecnico.csv', ',')

BP_EstadoRecursoTecnico = Carga_df('15_BP_EstadoRecursoTecnico.csv', ',')

TECNICA_RecursoTecnico = Carga_df('16_TECNICA_RecursoTecnico.csv', ',')

TECNICA_RecursosTecnicos_Servicios = Carga_df('17_TECNICA_RecursosTecnicos_Servicios.csv', ',')

BUSINESS_BOHistorico = Carga_df('18_BUSINESS_BOHistorico.csv', ',')

BP_TipoOperacion = Carga_df('19_BP_TipoOperacion.csv', ',')

TRAMITE_Operacion = Carga_df('20_TRAMITE_Operacion.csv', ',')

COMERCIAL_ProductoServicio = Carga_df('22_COMERCIAL_ProductoServicio.csv', ',')



In [5]:
def Ejecutar_Comando_mySql(Coneccion, Esquema, Comando):
    try:

        with Coneccion.cursor() as cursor:
                
            query = Comando                
            cursor.execute(query)

            Coneccion.commit()
            cursor.close()

        print(f"El comando: {Comando} fue ejecutado con éxito.")     

    except Error as e:
        print(f"Error al ejecutar el comando ({Comando}): {e}")      

In [11]:
def Cargar_Tabla_en_mySql(Coneccion, Esquema, NombreTabla, Datos):

    import numpy as np
    try:
        Datos_temp = Datos.replace(np.nan, None)
       
        with Coneccion.cursor() as cursor:
                
            query = f"SELECT * FROM {NombreTabla} LIMIT 1"
                
            cursor.execute(query)
            cursor.fetchall()
            results = cursor.column_names
            cols=""
            for e in results: 
                cols += e + ", " 
            cols=cols[:len(cols)-2]
                
            place = "%s, " * len(cols.split(", "))
            place = place [: len(place)-2]
                
            insert_query = f"""INSERT INTO {NombreTabla} ({cols}) VALUES ({place})"""
                 
            cursor.executemany(insert_query, Datos_temp.values.tolist())
            Coneccion.commit()
            cursor.close()

        print(f"Base de datos {NombreTabla} cargada con exito.")     

    except Error as e:
        print(f"Error Carga ({NombreTabla}): {e}")  
    

In [7]:
def Limpiar_Tabla_en_mySql(Coneccion, Esquema, NombreTabla):

    try:
        with Coneccion.cursor() as cursor:
              
            query = f"DELETE FROM {Esquema}.{NombreTabla}"
                                
            cursor.execute(query)
                                                    
            Coneccion.commit()
            cursor.close()
                
        print(f"Base de datos {Esquema}.{NombreTabla} borrada con exito.")     

    except Error as e:
        print(f"Error Limpiar ({Esquema}.{NombreTabla}): {e}")  

In [8]:
def Cargar_fotomes(Coneccion, inicioMes, finMes, desdeReclamos):

    try:

        with Coneccion.cursor() as cursor:
                
            query = f"""SELECT '{finMes}' AS foto_mes, cli.numeroCliente, s.id AS ServicioIPTV, cli.DISCRIMINATOR, \
            (SELECT newStateName FROM BUSINESS_BOHistorico boh WHERE boh.cuentacobro = cc.id \
             AND boh.SPDName = 'SPDCuentaCobro' AND date <= '{finMes}' ORDER BY boh.date desc, boh.id desc \
             LIMIT 1) AS EstadoCuenta, \
            bh.newStateName AS EstadoServicio, \
            (SELECT ccm.saldoAFecha  \
             FROM CTACTES_CuentaCobroMovimiento ccm \
             WHERE ccm.cuentaCobro = cc.id AND ccm.fechaRegistracion <= '{finMes}' \
             ORDER BY ccm.fechaMovimiento DESC, ccm.fechaRegistracion DESC, ccm.id desc \
             LIMIT 1) AS saldo, \
            cc.debitoAutomatico, \
            (SELECT COUNT(*) \
             FROM TRAMITE_Operacion t \
             WHERE t.servicio = s.id \
             AND t.fechaInicio BETWEEN '{desdeReclamos}' AND '{finMes}') AS cantidadReclamos, \
            s.fechaInstalacion, cli.fechaDesde AS FechaAltaCliente, s.puntoCentroGoogleMaps_x, s.puntoCentroGoogleMaps_y, \
            IFNULL((SELECT  1  \
                    FROM COMERCIAL_ProductoServicio ps \
                    INNER JOIN SERVICIOS_Servicio sub ON ps.servicioAbs = sub.id  \
                    INNER JOIN COMERCIAL_ProductoComercial pc ON ps.productocomercial = pc.id \
                    WHERE sub.servicioPadre = s.id  \
                    AND pc.descripcion LIKE '% HBO %' \
                    AND '{finMes}' BETWEEN ps.vigencia_desde AND IFNULL(ps.vigencia_hasta, NOW() ) LIMIT 1), 0) AS pqueteHbo, \
            IFNULL(( SELECT 1  \
                    FROM COMERCIAL_ProductoServicio ps \
                    INNER JOIN SERVICIOS_Servicio sub ON ps.servicioAbs = sub.id  \
                    INNER JOIN COMERCIAL_ProductoComercial pc ON ps.productocomercial = pc.id \
                    WHERE sub.servicioPadre = s.id  \
                    AND pc.descripcion LIKE '% HD %' \
                    AND '{finMes}' BETWEEN ps.vigencia_desde AND IFNULL(ps.vigencia_hasta, NOW() ) LIMIT 1), 0) AS paqueteHD, \
            IFNULL((SELECT 1  \
                    FROM COMERCIAL_ProductoServicio ps \
                    INNER JOIN SERVICIOS_Servicio sub ON ps.servicioAbs = sub.id  \
                    INNER JOIN COMERCIAL_ProductoComercial pc ON ps.productocomercial = pc.id \
                    WHERE sub.servicioPadre = s.id  \
                    AND pc.descripcion LIKE '% adultos %'  \
                    AND '{finMes}' BETWEEN ps.vigencia_desde AND IFNULL(ps.vigencia_hasta, NOW() ) LIMIT 1), 0) AS paqueteAdultos, \
            IFNULL((SELECT 1  \
                    FROM COMERCIAL_ProductoServicio ps \
                    INNER JOIN SERVICIOS_Servicio sub ON ps.servicioAbs = sub.id  \
                    INNER JOIN COMERCIAL_ProductoComercial pc ON ps.productocomercial = pc.id \
                    WHERE sub.servicioPadre = s.id  \
                    AND pc.descripcion LIKE '% FUTBOL %' \
                    AND '{finMes}' BETWEEN ps.vigencia_desde AND IFNULL(ps.vigencia_hasta, NOW() ) LIMIT 1), 0) AS PaqueteFutbol, \
            IFNULL((SELECT  1  \
                    FROM COMERCIAL_ProductoServicio ps \
                    INNER JOIN SERVICIOS_Servicio sub ON ps.servicioAbs = sub.id  \
                    INNER JOIN COMERCIAL_ProductoComercial pc ON ps.productocomercial = pc.id \
                    WHERE sub.servicioPadre = s.id  \
                    AND pc.descripcion LIKE 'bonifi%' \
                    AND '{finMes}' BETWEEN ps.vigencia_desde AND IFNULL(ps.vigencia_hasta, NOW() )), 0) AS TieneBonificacion, \
            (SELECT COUNT(*) \
             FROM SERVICIOS_Servicio s1 \
             INNER JOIN COMERCIAL_ProductoServicio ps ON ps.servicioAbs = s1.id \
             WHERE s1.id <> s.id AND s1.SeccionManzana = s.SeccionManzana \
             AND '{finMes}' BETWEEN ps.vigencia_desde AND IFNULL(ps.vigencia_hasta, NOW() ) \
             AND s1.cuentaCobro = s.cuentaCobro) AS cantidadOtrosServiciosMismoLote, \
            (SELECT COUNT(*) \
             FROM SERVICIOS_Servicio s1 \
             INNER JOIN COMERCIAL_ProductoServicio ps ON ps.servicioAbs = s1.id \
             WHERE s1.id <> s.id AND s1.SeccionManzana <> s.SeccionManzana \
             AND '{finMes}' BETWEEN ps.vigencia_desde AND IFNULL(ps.vigencia_hasta, NOW() ) \
             AND s1.cuentaCobro = s.cuentaCobro) AS cantidadOtrosServiciosOtroLote \
            FROM CLI_Cliente cli  \
            INNER JOIN CTACTES_CuentaCobro cc ON cc.cliente = cli.id \
            INNER JOIN SERVICIOS_Servicio s ON s.cuentaCobro = cc.id \
            INNER JOIN BP_TipoServicio ts ON s.tipoServicio = ts.id AND ts.codigo IN ('IPTV', 'OTTV') \
            INNER JOIN BP_EstadoCliente ecli ON ecli.id = cli.estado \
            INNER JOIN BUSINESS_BOHistorico bh ON  bh.servicio = s.id  \
            INNER JOIN BP_EstadoServicio es ON bh.newStateName = es.descripcion AND es.Activo = 1 \
            WHERE bh.id = (SELECT  boh1.id \
                            FROM BUSINESS_BOHistorico boh1 \
                            WHERE boh1.servicio = s.id AND boh1.date <= '{finMes}' \
                            ORDER BY boh1.id desc \
                            LIMIT 1)"""
           
            
            cursor.execute(query)
            data = cursor.fetchall()
            
            Coneccion.commit()
            cursor.close()

        data2 = pd.DataFrame(data)
        
        print(f"Cargo foto mes del {finMes}.")     

    except Error as e:
        print(f"Error Carga : {e}")  
    
    return data2;
    

In [ ]:
try:
    with connect(
        host="localhost",
        user="root",
        password="UADE2021",
        database = "FINAL"
    ) as connection:
        
        Limpiar_Tabla_en_mySql(connection, "FINAL", "COMERCIAL_ProductoServicio")    
        Limpiar_Tabla_en_mySql(connection, "FINAL", "TRAMITE_Operacion")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_TipoOperacion")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BUSINESS_BOHistorico")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "TECNICA_RecursosTecnicos_Servicios")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "TECNICA_RecursoTecnico")        
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoRecursoTecnico")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_TipoRecursoTecnico")        
        Limpiar_Tabla_en_mySql(connection, "FINAL", "SERVICIOS_Servicio")        
        Limpiar_Tabla_en_mySql(connection, "FINAL", "COMERCIAL_ProductoComercial")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "CTACTES_CuentaCobroMovimiento")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_TipoMovimiento")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "CTACTES_CuentaCobro")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_Moneda")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoServicio_BPTipoServicio")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_TipoServicio")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoServicio")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoCuentaCobro")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoTipoCtaCobro")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_TipoCuentaCobro")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "CLI_Cliente")
        Limpiar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoCliente")
         
except Error as e:
    connection.close()
    print(f"Error: {e}")         

In [ ]:
try:
    with connect(
        host="localhost",
        user="root",
        password="UADE2021",
        database = "FINAL"
    ) as connection:
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoCliente", BP_EstadoCliente)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "CLI_Cliente", CLI_Cliente)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_TipoCuentaCobro", BP_TipoCuentaCobro)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoTipoCtaCobro", BP_EstadoTipoCtaCobro)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoCuentaCobro", BP_EstadoCuentaCobro)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoServicio", BP_EstadoServicio)
                
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_TipoServicio", BP_TipoServicio)
                
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoServicio_BPTipoServicio", BP_EstadoServicio_BPTipoServicio)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_Moneda", BP_Moneda)
       
        Cargar_Tabla_en_mySql(connection, "FINAL", "CTACTES_CuentaCobro", CTACTES_CuentaCobro)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_TipoMovimiento", BP_TipoMovimiento)
       
        Cargar_Tabla_en_mySql(connection, "FINAL", "CTACTES_CuentaCobroMovimiento", CTACTES_CuentaCobroMovimiento)
                
        Cargar_Tabla_en_mySql(connection, "FINAL", "COMERCIAL_ProductoComercial", COMERCIAL_ProductoComercial)
        
        comando = "ALTER TABLE SERVICIOS_Servicio DROP CONSTRAINT FK_SERVICIOS_ServicioPadre_Servicio"
        Ejecutar_Comando_mySql(connection, "FINAL", comando)        
        Cargar_Tabla_en_mySql(connection, "FINAL", "SERVICIOS_Servicio", SERVICIOS_Servicio)
        comando = """ALTER TABLE SERVICIOS_Servicio
                    ADD CONSTRAINT FK_SERVICIOS_ServicioPadre_Servicio FOREIGN KEY
                    (
                    servicioPadre
                    )
                    references
                    SERVICIOS_Servicio
                    (
                    id
                    );
                 """
        Ejecutar_Comando_mySql(connection, "FINAL", comando)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_TipoRecursoTecnico", BP_TipoRecursoTecnico)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_EstadoRecursoTecnico", BP_EstadoRecursoTecnico)

        comando = "ALTER TABLE TECNICA_RecursoTecnico DROP CONSTRAINT FK_TECNICA_RecursoTecnico_RecursoContenedor"
        Ejecutar_Comando_mySql(connection, "FINAL", comando)
        Cargar_Tabla_en_mySql(connection, "FINAL", "TECNICA_RecursoTecnico", TECNICA_RecursoTecnico)
        comando = """ALTER TABLE TECNICA_RecursoTecnico 
                    ADD CONSTRAINT FK_TECNICA_RecursoTecnico_RecursoContenedor FOREIGN KEY
                    (
                    recursoContenedor
                    )
                    references
                    TECNICA_RecursoTecnico
                    (
                    id
                    );
                """
        Ejecutar_Comando_mySql(connection, "FINAL", comando)
                
        Cargar_Tabla_en_mySql(connection, "FINAL", "TECNICA_RecursosTecnicos_Servicios", TECNICA_RecursosTecnicos_Servicios)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BUSINESS_BOHistorico", BUSINESS_BOHistorico)
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "BP_TipoOperacion", BP_TipoOperacion)
        
                
        Cargar_Tabla_en_mySql(connection, "FINAL", "TRAMITE_Operacion", TRAMITE_Operacion)
                
        
        Cargar_Tabla_en_mySql(connection, "FINAL", "COMERCIAL_ProductoServicio", COMERCIAL_ProductoServicio)    
        
except Error as e:
    connection.close()
    print(f"Error: {e}") 

In [ ]:
try:
    with connect(
        host="localhost",
        user="root",
        password="UADE2021",
        database = "FINAL"
    ) as connection:
        
        dataMayo = Cargar_fotomes(connection, '2021-05-01', '2021-05-31 23:59:59', '2020-12-01')
        dataJunio = Cargar_fotomes(connection, '2021-06-01', '2021-06-30 23:59:59', '2021-01-01')
        dataJulio = Cargar_fotomes(connection, '2021-07-01', '2021-07-31 23:59:59', '2021-02-01')
        dataAgosto = Cargar_fotomes(connection, '2021-08-01', '2021-08-30 23:59:59', '2021-03-01')
        dataSeptiembre = Cargar_fotomes(connection, '2021-09-01', '2021-09-30 23:59:59', '2021-04-01')
        dataOctubre = Cargar_fotomes(connection, '2021-10-01', '2021-10-31 23:59:59', '2021-05-01')
        dataNoviembre = Cargar_fotomes(connection, '2021-11-01', '2021-11-30 23:59:59', '2021-06-01')
    
        frames = [dataMayo, dataJunio, dataJulio, dataAgosto, dataSeptiembre, dataOctubre, dataNoviembre ]
        fotomes = pd.concat(frames)
        
        fotomes.to_csv('foto_mes_281122.csv', index=False)
        
except Error as e:
    connection.close()
    print(f"Error: {e}")    
    
    